### Errors
- batch norm during encoding?
- not masking the pad inputs for backprop


# Things you're trying and may need to change back
- batchnorm eval on cnn during training
- tanh non-linearity when projecting cnn output to embedding space
- using cnn output as first input to rnn instead of as hidden state -- what about SOS?